# [실습 프로젝트]

- **다음과 같은 요구사항을 Gradio ChatInterface로 구현합니다**

- 주제: 맞춤형 여행 일정 계획 어시스턴트
- 기능: 
   - OpenAI Chat Completion API와 LangChain을 활용하여 사용자의 선호도에 맞는 여행 일정을 생성
   - LCEL을 사용하여 단계별 프롬프트 체인 구성 (사용자 입력 분석 -> 일정 생성 -> 세부 계획 수립)
   - 채팅 히스토리 사용하여 답변 생성
   - Gradio 인터페이스를 통해 사용자와 대화형으로 상호작용

- 주요 포인트:

   1. **모델 매개변수 최적화**
      - temperature=0.7: 적당한 창의성을 유지하면서 일관된 응답 생성
      - top_p=0.9: 높은 확률의 토큰만 선택하여 응답의 품질 향상
      - presence_penalty와 frequency_penalty: 반복적인 응답을 줄이고 다양한 제안 생성

   2. **시스템 프롬프트 설계**
      - 여행 플래너로서의 역할과 응답 가이드라인을 명확히 정의
      - 구체적인 정보를 포함하도록 지시
      - 한국어 응답 명시

   3. **메모리 관리**
      - Gradio 또는 LangChain 메모리 기능을 사용하여 대화 컨텍스트 유지
      - 이전 대화 내용을 바탕으로 연속성 있는 응답 생성

# Setting

In [ ]:
from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_core.messages import HumanMessage, AIMessage
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnableParallel

In [20]:
from IPython.display import display, Markdown

In [2]:
import gradio as gr

/Users/sunhwaryu/Documents/llm-study/prj01/.venv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [1]:
from dotenv import load_dotenv
load_dotenv()

True

# Format

- Input : 일정(날짜, 일수 등), 여행 장소 
- Output : 여행 기간 동안 장소별 세부 계획
- Format :

| 날짜 및 시간 | 장소 | 세부 계획 |
|---|---|---|
| Time 1 | 장소 A | - |
| Time 2 | 장소 B | - |
| Time 3 | 장소 C | - |
| Time 4 | 장소 D | - |


### 예시

- 여행 일정 : 오늘로부터 2박 3일
- 교통편 일정 : 
    - 오늘 기차 오전 8시 서울역 출발
    - 3일 후 오후 5시 부산역 출발
- 여행 장소 : 부산

| 날짜 및 시간 | 장소 | 세부 계획 |
|---|---|---|
| 2025-05-01 18:00 | 

# Prompt / Model Test

In [23]:
summarize_templete = """
{text}

위에 입력된 텍스트를 다음 항목으로 요약해주세요: 
- 여행 일정 :
- 교통편 일정 :
- 여행 장소 :
- 여행 스타일 :
- 예산 :
- 추천 숙소 :"""

summarize_prompt = ChatPromptTemplate.from_messages([
    ("system", "당신은 여행 일정 작성을 도와주는 AI 어시스턴트입니다."),
    # MessagesPlaceholder("chat_history"),
    ("human", summarize_templete)
])

In [24]:
planner_prompt = ChatPromptTemplate.from_template("""
다음 텍스트의 여행 일정을 기반으로 세부 여행 일정을 짜주세요.
텍스트: {summary}
규칙:
1. 날짜 및 시간과 장소, 세부 계획 항목으로 표 형태로 작성하세요.
2. 여행 스타일과 추천 숙소, 예산에 맞추어 동선을 고려하여 장소를 추천하세요.
답변:""")

In [54]:
# 문자열 출력 파서
output_parser = StrOutputParser()

# 체인 구성
model = ChatOpenAI(
    model="gpt-4.1-mini",
    temperature=0.4,
    top_p=0.7
)

# 요약 체인
summarize_chain = summarize_prompt | model 

# 감정 분석 체인
planner_chain = planner_prompt | model | output_parser

# 전체 체인
chain = (
    summarize_chain 
    | RunnableParallel(
        summary=lambda x: x.content,
        plan=lambda x: planner_chain.invoke({"summary": x.content}),
    )
)

In [55]:
text = """내일 오전 8시에 서울역에서 출발해서 오전 11시에 부산역에 도착해.
2박 3일동안 부산 기장군 부근에서 여행하고 싶어.
맛있는 거 먹으면서 돌아다니고 싶고, 명소도 가고 싶어.
그런데 자동차가 없어서 걸어다니거나 대중교통을 이용해야해.
그리고 여행 마지막 날은 오후 5시에 부산역에서 출발해.
여동생이랑 둘이서 가려고 하고, 예산은 50만원 내외로 부탁해."""

In [56]:
result = chain.invoke({"text": text})
print(f"<요약>")
display(Markdown(result['summary']))

print(f"<일정>")
display(Markdown(result['plan']))

<요약>


- 여행 일정 : 2박 3일, 내일 오전 8시 서울역 출발, 오전 11시 부산역 도착, 마지막 날 오후 5시 부산역 출발  
- 교통편 일정 : 서울역에서 부산역까지 기차 이동, 부산 내에서는 도보 및 대중교통 이용  
- 여행 장소 : 부산 기장군 부근  
- 여행 스타일 : 맛집 탐방과 명소 방문, 도보 및 대중교통 활용  
- 예산 : 50만원 내외 (2인 기준)  
- 추천 숙소 : 기장군 내 대중교통 접근성이 좋은 게스트하우스 또는 중저가 호텔

<일정>


| 날짜       | 시간          | 장소/활동                      | 세부 계획 및 비고                                      |
|------------|---------------|-------------------------------|-------------------------------------------------------|
| 1일차 (내일) | 08:00         | 서울역 출발                   | KTX 탑승, 2인 기준 약 10만원 예상                     |
|            | 11:00         | 부산역 도착                   | 부산역 도착 후 기장행 버스 또는 지하철 환승            |
|            | 12:00~13:30   | 점심 - 기장시장 내 해산물 맛집 | 신선한 회 또는 해산물 백반 추천, 2인 약 3~4만원 예상   |
|            | 14:00~16:00   | 오랑대 해변 방문              | 도보 산책 및 사진 촬영, 자연경관 감상                  |
|            | 16:30~18:00   | 기장군 대중교통 접근성 좋은 숙소 체크인 | 게스트하우스 또는 중저가 호텔 (1박 약 5~7만원)          |
|            | 18:30~20:00   | 저녁 - 기장 유명 돼지국밥 맛집 | 현지 인기 맛집 방문, 2인 약 2~3만원 예상               |
|            | 20:00~21:30   | 기장읍내 산책 및 카페 방문    | 분위기 좋은 카페에서 휴식 및 디저트                      |
| 2일차       | 08:00~09:00   | 아침 식사 - 숙소 인근 카페    | 간단한 브런치 또는 커피                                 |
|            | 09:30~12:00   | 해동용궁사 방문               | 대중교통 이용, 부산 대표 명소, 입장료 무료             |
|            | 12:30~14:00   | 점심 - 기장군 내 유명 밀면집 | 부산식 밀면 또는 국수, 2인 약 2만원 예상                |
|            | 14:30~16:30   | 기장 죽성성당 및 죽성 해변 방문 | 도보 산책 및 사진 촬영                                 |
|            | 17:00~18:30   | 숙소 복귀 및 휴식             |                                                       |
|            | 19:00~20:30   | 저녁 - 기장군 내 해산물 BBQ 또는 조개구이 맛집 | 2인 약 3~4만원 예상                                    |
|            | 21:00~22:00   | 숙소 근처 산책 또는 휴식      |                                                       |
| 3일차       | 08:00~09:00   | 아침 식사 - 숙소 인근 카페    |                                                       |
|            | 09:30~11:30   | 기장 전통시장 방문 및 간식 탐방 | 다양한 길거리 음식 시식, 기념품 구매 가능               |
|            | 12:00~13:30   | 점심 - 기장군 내 유명 분식집 | 떡볶이, 순대 등 부산식 분식, 2인 약 1.5~2만원 예상      |
|            | 14:00~15:30   | 장안사 방문                  | 대중교통 이용, 조용한 사찰 방문 및 산책                  |
|            | 16:00~16:30   | 부산역 이동                   | 버스 또는 지하철 이용                                  |
|            | 17:00         | 부산역 출발                   | KTX 탑승, 서울역으로 귀환                              |

---

### 예산 요약 (2인 기준, 대략적)
- 교통비 (서울-부산 KTX 왕복): 약 20만원
- 숙박비 (2박): 약 10~14만원
- 식비 (6끼): 약 12~15만원
- 기타 (대중교통, 입장료, 간식 등): 약 3~5만원  
총 예상 비용: 약 45~54만원 내외 (예산 50만원 내외 적합)

---

### 숙소 추천
- 기장군 내 대중교통 접근성이 좋은 게스트하우스 또는 중저가 호텔 (예: 기장역 근처 게스트하우스, 기장읍내 호텔)
- 예약 시 조식 포함 여부 확인 권장

---

### 동선 및 여행 스타일 반영
- 맛집 탐방 위주로 점심, 저녁 식사 장소 선정
- 도보 및 대중교통으로 이동 가능한 명소 위주로 일정 구성
- 자연 경관과 문화 명소를 균형 있게 방문하도록 계획

# Gradio ChatInterface

In [50]:
def get_chain():
    summarize_templete = """
    {text}

    위에 입력된 텍스트를 다음 항목으로 요약해주세요: 
    - 여행 일정 :
    - 교통편 일정 :
    - 여행 장소 :
    - 여행 스타일 :
    - 예산 :
    - 추천 숙소 :"""

    summarize_prompt = ChatPromptTemplate.from_messages([
        ("system", "당신은 여행 일정 작성을 도와주는 AI 어시스턴트입니다."),
        MessagesPlaceholder("chat_history"),
        ("human", summarize_templete)
    ])

    planner_prompt = ChatPromptTemplate.from_template("""
    다음 텍스트의 여행 일정을 기반으로 세부 여행 일정을 짜주세요.
    텍스트: {summary}
    규칙:
    1. 날짜 및 시간과 장소, 세부 계획 항목으로 표 형태로 작성하세요.
    2. 여행 스타일과 추천 숙소, 예산에 맞추어 동선을 고려하여 장소를 추천하세요.
    답변:""")

    # 문자열 출력 파서
    output_parser = StrOutputParser()

    # 체인 구성
    model = ChatOpenAI(
        model="gpt-4.1",
        temperature=0.4,
        top_p=0.7
    )

    # 요약 체인
    summarize_chain = summarize_prompt | model 

    # 감정 분석 체인
    planner_chain = planner_prompt | model | output_parser

    # 전체 체인
    chain = (
        summarize_chain 
        | RunnableParallel(
            summary=lambda x: x.content,
            plan=lambda x: planner_chain.invoke({"summary": x.content}),
        )
    )
    return chain

In [51]:
def answer_invoke(message, history):
    chain = get_chain()
    history_messages = []
    for msg in history:
        if msg['role'] == "user":
            history_messages.append(HumanMessage(content=msg['content']))
        elif msg['role'] == "assistant":
            history_messages.append(AIMessage(content=msg['content']))

    response = chain.invoke({
        "chat_history": history_messages,
        "text": message
    })
    response = f"<요약>\n{response['summary']}\n\n<일정>\n{response['plan']}"
    return response

In [53]:
demo = gr.ChatInterface(
    fn=answer_invoke,         # 메시지 처리 함수
    title="여행 일정 어시스턴트", # 채팅 인터페이스의 제목
    type="messages"
)

# Gradio 인터페이스 실행
demo.launch()

* Running on local URL:  http://127.0.0.1:7861
* To create a public link, set `share=True` in `launch()`.


In [52]:
demo.close()

Closing server running on port: 7861
